# XGBoost Model

This file aims to train and test an XGBoost model on the data.

In [15]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid

# Preparing the data

In [16]:
df = pd.read_csv('all_hourly_data.csv')

In [17]:
print(df.shape)
df = df.drop(['date'], axis=1) #we're adding dummies later for this

(63689, 25)


Let's generate lags for

- The previous 24 hours (i.e., lags 1 to 24),
- The same hour 2 and 3 days ago (lags 48 and 72),
- The same hour 1 week ago (lag 168),

Let's also generate hourly, 2-3 days, and weekly dummies.

In [18]:
#handle the time column
df['interval_start_local'] = pd.to_datetime(df['interval_start_local'], utc=True)
df = df.set_index('interval_start_local')

#lagging 
cols_to_lag = df.columns
lags = list(range(1, 25)) + [48, 72, 168]

lagged_frames = []
for lag in lags:
    lagged = df[cols_to_lag].shift(lag)
    lagged.columns = [f"{col}_lag{lag}" for col in cols_to_lag]
    lagged_frames.append(lagged)

lagged_df = pd.concat(lagged_frames, axis=1)
df_with_lags = pd.concat([df, lagged_df], axis=1).reset_index()
df_with_lags = df_with_lags.copy()

#Lose the first 168 observations because our largest lag is 1 week long
df_with_lags = df_with_lags.iloc[168:].reset_index(drop=True)


In [19]:
df_with_lags['hour'] = pd.to_datetime(df_with_lags['interval_start_local']).dt.hour
df_with_lags['dayofweek'] = pd.to_datetime(df_with_lags['interval_start_local']).dt.dayofweek

#one-hot encode dummies
hour_dummies = pd.get_dummies(df_with_lags['hour'], prefix='hour')
dow_dummies = pd.get_dummies(df_with_lags['dayofweek'], prefix='dow')
hour_dummies = hour_dummies.astype(int)
dow_dummies = dow_dummies.astype(int)

#merge
df = pd.concat([df_with_lags, hour_dummies, dow_dummies], axis=1)
df = df.drop(['hour', 'dayofweek'], axis=1)

print(df.head())

       interval_start_local  north_x_load  south_load  west_x_load  \
0 2018-01-08 06:00:00+00:00      11292.28     7720.14      3626.51   
1 2018-01-08 07:00:00+00:00      11134.52     7540.80      3615.65   
2 2018-01-08 08:00:00+00:00      11183.56     7529.38      3624.71   
3 2018-01-08 09:00:00+00:00      11497.37     7687.16      3678.35   
4 2018-01-08 10:00:00+00:00      12203.96     8053.77      3761.58   

   houston_load  total_load  lmp_HB_BUSAVG  coal_and_lignite      hydro  \
0       8181.37    30820.31      18.535000       9942.516942  39.045263   
1       7980.93    30271.90      18.107500      10059.864300  15.672631   
2       7866.77    30204.42      18.019167       9912.115128  14.312730   
3       7874.74    30737.62      18.026667       9731.137220  14.721392   
4       8095.04    32114.36      18.192500       9950.836098  15.266061   

       nuclear  ...  hour_21  hour_22  hour_23  dow_0  dow_1  dow_2  dow_3  \
0  5123.325668  ...        0        0        0    

In [7]:
#'week of the year' dummies

df['weekofyear'] = pd.to_datetime(df['interval_start_local']).dt.isocalendar().week

weekofyear_dummies = pd.get_dummies(df['weekofyear'], prefix='weekofyear', drop_first=False)
weekofyear_dummies = weekofyear_dummies.astype(int)

df = pd.concat([df, weekofyear_dummies], axis=1)
df = df.drop(['weekofyear'], axis=1)

# Preview result
print(df.head())

       interval_start_local  north_x_load  south_load  west_x_load  \
0 2018-01-08 06:00:00+00:00      11292.28     7720.14      3626.51   
1 2018-01-08 07:00:00+00:00      11134.52     7540.80      3615.65   
2 2018-01-08 08:00:00+00:00      11183.56     7529.38      3624.71   
3 2018-01-08 09:00:00+00:00      11497.37     7687.16      3678.35   
4 2018-01-08 10:00:00+00:00      12203.96     8053.77      3761.58   

   houston_load  total_load  lmp_HB_BUSAVG  coal_and_lignite      hydro  \
0       8181.37    30820.31      18.535000       9942.516942  39.045263   
1       7980.93    30271.90      18.107500      10059.864300  15.672631   
2       7866.77    30204.42      18.019167       9912.115128  14.312730   
3       7874.74    30737.62      18.026667       9731.137220  14.721392   
4       8095.04    32114.36      18.192500       9950.836098  15.266061   

       nuclear  ...  weekofyear_44  weekofyear_45  weekofyear_46  \
0  5123.325668  ...              0              0           

In [8]:
#drop one dummy for each category to prevent multicollinearity
df = df.drop(['weekofyear_53', 'dow_6', 'hour_23'], axis=1)


We could drop lagged/transformed versions of the target variable but keeping them in allows for an autoregressive model, which is desirable for maximum accuracy.

In [9]:
print(df.columns.tolist())
print(df.shape)

['interval_start_local', 'north_x_load', 'south_load', 'west_x_load', 'houston_load', 'total_load', 'lmp_HB_BUSAVG', 'coal_and_lignite', 'hydro', 'nuclear', 'power_storage', 'solar', 'wind', 'natural_gas', 'other_gen', 'coast_temp', 'east_temp', 'far_west_temp', 'north_y_temp', 'north_central_temp', 'south_central_temp', 'southern_temp', 'west_y_temp', 'hour_lag1', 'north_x_load_lag1', 'south_load_lag1', 'west_x_load_lag1', 'houston_load_lag1', 'total_load_lag1', 'lmp_HB_BUSAVG_lag1', 'coal_and_lignite_lag1', 'hydro_lag1', 'nuclear_lag1', 'power_storage_lag1', 'solar_lag1', 'wind_lag1', 'natural_gas_lag1', 'other_gen_lag1', 'coast_temp_lag1', 'east_temp_lag1', 'far_west_temp_lag1', 'north_y_temp_lag1', 'north_central_temp_lag1', 'south_central_temp_lag1', 'southern_temp_lag1', 'west_y_temp_lag1', 'hour_lag2', 'north_x_load_lag2', 'south_load_lag2', 'west_x_load_lag2', 'houston_load_lag2', 'total_load_lag2', 'lmp_HB_BUSAVG_lag2', 'coal_and_lignite_lag2', 'hydro_lag2', 'nuclear_lag2', 'p

In [10]:
#print the min and max of the time labels
min_time = df['interval_start_local'].min()
max_time = df['interval_start_local'].max()

min_time, max_time

(Timestamp('2018-01-08 06:00:00+0000', tz='UTC'),
 Timestamp('2025-04-13 04:00:00+0000', tz='UTC'))

63521 individual observations are very promising—a large sample size bodes well for performance in this model. Let's generate our train/val/test split (which must include sequential data, due to time dependencies):

- Train: 2018 (beginning on week 2) - 2022
- Val: 2022-Jun 2023
- Test: Jun 2023-Apr 2025 (present)

In [11]:
#define date ranges for each set
train_start = pd.Timestamp('2018-01-08 06:00:00+0000', tz='UTC')
train_end = pd.Timestamp('2022-12-31 23:59:59+0000', tz='UTC')

val_start = pd.Timestamp('2022-01-01 00:00:00+0000', tz='UTC')
val_end = pd.Timestamp('2023-06-01 00:00:00+0000', tz='UTC')

test_start = pd.Timestamp('2023-06-01 00:00:00+0000', tz='UTC')
test_end = pd.Timestamp('2025-04-13 04:00:00+0000', tz='UTC')

#filter
train_data = df[(df['interval_start_local'] >= train_start) & (df['interval_start_local'] <= train_end)]
val_data = df[(df['interval_start_local'] >= val_start) & (df['interval_start_local'] <= val_end)]
test_data = df[(df['interval_start_local'] >= test_start) & (df['interval_start_local'] <= test_end)]

#check dims
print(f"Train set size: {train_data.shape[0]}")
print(f"Validation set size: {val_data.shape[0]}")
print(f"Test set size: {test_data.shape[0]}")

Train set size: 43650
Validation set size: 12385
Test set size: 16247


In [12]:
#drop the time variable
train_data = train_data.drop(['interval_start_local'], axis=1)
val_data = val_data.drop(['interval_start_local'], axis=1)
test_data = test_data.drop(['interval_start_local'], axis=1)

#identify non-dummy columns
cols_to_scale = [col for col in train_data.columns if 'hour' not in col and 'dow' not in col and 'weekofyear' not in col]

#fit scaler to training data
scaler = StandardScaler()
train_data_scaled = train_data.copy()
train_data_scaled[cols_to_scale] = scaler.fit_transform(train_data[cols_to_scale])

#apply scaler to val/test data
val_data_scaled = val_data.copy()
val_data_scaled[cols_to_scale] = scaler.transform(val_data[cols_to_scale])
test_data_scaled = test_data.copy()
test_data_scaled[cols_to_scale] = scaler.transform(test_data[cols_to_scale])

#preview to check
print("Scaled training data preview:\n", train_data_scaled[cols_to_scale].head())
print("Scaled validation data preview:\n", val_data_scaled[cols_to_scale].head())
print("Scaled test data preview:\n", test_data_scaled[cols_to_scale].head())

Scaled training data preview:
    north_x_load  south_load  west_x_load  houston_load  total_load  \
0     -1.095496   -1.373361    -1.809739     -1.436398   -1.397223   
1     -1.132904   -1.434522    -1.822247     -1.512426   -1.451934   
2     -1.121276   -1.438417    -1.811812     -1.555728   -1.458666   
3     -1.046865   -1.384608    -1.750034     -1.552705   -1.405472   
4     -0.879318   -1.259581    -1.654176     -1.469144   -1.268123   

   lmp_HB_BUSAVG  coal_and_lignite     hydro   nuclear  power_storage  ...  \
0      -0.103216          0.452893 -0.512432  0.703447      -0.067082  ...   
1      -0.105110          0.498833 -0.853273  0.699100      -0.067082  ...   
2      -0.105501          0.440991 -0.873105  0.698724      -0.067082  ...   
3      -0.105468          0.370141 -0.867145  0.697035      -0.067082  ...   
4      -0.104733          0.456150 -0.859202  0.699385      -0.067082  ...   

   natural_gas_lag168  other_gen_lag168  coast_temp_lag168  east_temp_lag168  \

# Tuning the model



In [13]:
#Define y vectors 
X_train = train_data_scaled.drop('lmp_HB_BUSAVG', axis=1)
y_train = train_data_scaled['lmp_HB_BUSAVG']
X_val = val_data_scaled.drop('lmp_HB_BUSAVG', axis=1)
y_val = val_data_scaled['lmp_HB_BUSAVG']
X_test = test_data_scaled.drop('lmp_HB_BUSAVG', axis=1)
y_test = test_data_scaled['lmp_HB_BUSAVG']

In [14]:
#Define parameter grid to search over (TEST)
param_grid = {
    'learning_rate': [0.01, 0.05],
    'n_estimators': [500],
    'max_depth': [3, 10],
    'subsample': [0.7, 0.95],
    'colsample_bytree': [0.8, 1.0]
}

In [20]:
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric='rmse')
results = []
i = 0

for params in ParameterGrid(param_grid):
    xgb_model.set_params(**params) 
    xgb_model.fit(X_train, y_train)

    y_val_pred = xgb_model.predict(X_val) #rather than CV, make predictions on val set
    mse = mean_squared_error(y_val, y_val_pred)
    rmse = mse ** 0.5
    results.append({'params': params, 'rmse': rmse})

    i += 1
    if i % 10 == 0:
        print(f"Trained {i}/{len(ParameterGrid(param_grid))} models. Moving to next.")

#Determine best model
best_result = min(results, key=lambda x: x['rmse'])
print(f"Best parameters: {best_result['params']}")
print(f"Best validation RMSE: {best_result['rmse']}")

Trained 10/16 models. Moving to next.
Best parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 500, 'subsample': 0.95}
Best validation RMSE: 0.04855397022649469
